In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

StatementMeta(PerforTestSpark, 0, 1, Finished, Available)

In [2]:
import tensorflow as tf
import gensim
import nltk
import re
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

StatementMeta(PerforTestSpark, 0, 2, Finished, Available)

In [3]:
df = pd.read_csv('abfss://ic3insightdev@ic3insightdev.dfs.core.windows.net/SparkLogs/movie.csv')
print(df.tail(5))

StatementMeta(PerforTestSpark, 0, 3, Finished, Available)

                                                    text  label
39995  "Western Union" is something of a forgotten cl...      1
39996  This movie is an incredible piece of work. It ...      1
39997  My wife and I watched this movie because we pl...      0
39998  When I first watched Flatliners, I was amazed....      1
39999  Why would this film be so good, but only gross...      1

In [4]:
target = df.pop('label')

StatementMeta(PerforTestSpark, 0, 4, Finished, Available)

In [5]:
raw_ds=tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

StatementMeta(PerforTestSpark, 0, 5, Finished, Available)

In [6]:
raw_ds = raw_ds.batch(16)

StatementMeta(PerforTestSpark, 0, 6, Finished, Available)

In [7]:
raw_ds

StatementMeta(PerforTestSpark, 0, 7, Finished, Available)

<BatchDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
total=int(40000/16)
print(total)
skip=int(8000/16)
train=int(22400/16)
ds_raw_shuffled = raw_ds.shuffle(total, reshuffle_each_iteration=False)

ds_test = ds_raw_shuffled.take(skip)
ds_train_valid = ds_raw_shuffled.skip(skip)
ds_train = ds_train_valid.take(train)
ds_valid = ds_train_valid.skip(train)

StatementMeta(PerforTestSpark, 0, 8, Finished, Available)

2500

In [9]:
for text_batch, label_batch in ds_train.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]}')
        label = label_batch.numpy()[i]
        print(f'Label : {label}')

StatementMeta(PerforTestSpark, 0, 9, Finished, Available)

Review: [b"This film's premise seems to be that the passing of the World War 2 generation in America, with its apple-pie phoniness and hypocritical morality, was a terrible tragedy. Those awful hippies ruined everything apparently.<br /><br />What holds the film together are the excellent performances - particularly Lemmon's which is truly remarkable. Otherwise we have a boring slice-of-life drama (just over 24 hours of Jack's life) with pretensions.<br /><br />I found it a chore to sit through."]
Label : 0
Review: [b"I found this film to be a fascinating study of a family in crisis. When Leo, the oldest announces that he is HIV+ the reactions of the family members alone and with each other was touching and yet strange.<br /><br />I have never seen a family that was as physically demonstrative as this one; nor one as likely to shout at each other. I didn't understand why the family felt that youngest couldn't deal with the news but once past that difficult I found this a thoroughly mov

In [10]:
bert_model_name = 'talking-heads_base'
map_name_to_handle = 'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/2'
map_model_to_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

tfhub_handle_encoder = map_name_to_handle
tfhub_handle_preprocess = map_model_to_preprocess

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

StatementMeta(PerforTestSpark, 0, 10, Finished, Available)

BERT model selected           : https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/2
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3

In [11]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

StatementMeta(PerforTestSpark, 0, 11, Finished, Available)

In [12]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

StatementMeta(PerforTestSpark, 0, 12, Finished, Available)

In [13]:
train_data=ds_train
valid_data=ds_valid
test_data=ds_test

StatementMeta(PerforTestSpark, 0, 13, Finished, Available)

In [14]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    pooled_net = outputs['pooled_output']
    sequence_output = outputs["sequence_output"]
    net = tf.keras.layers.Dropout(0.5)(pooled_net)
    net = tf.keras.layers.Dense(1, activation='tanh', name='classifier')(net)
    return tf.keras.Model(text_input, net)

StatementMeta(PerforTestSpark, 0, 14, Finished, Available)

In [15]:
classifier_model = build_classifier_model()
text_test = ['this movie is great...']
text_preprocessed = bert_preprocess_model(text_test)
text_test2 = ['This Movie Is Great.']
text_preprocessed2 = bert_preprocess_model(text_test2)
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Word Ids   : {text_preprocessed2["input_word_ids"][0, :12]}')

StatementMeta(PerforTestSpark, 0, 15, Finished, Available)

Word Ids   : [ 101 2023 3185 2003 2307 1012 1012 1012  102    0    0    0]
Word Ids   : [ 101 2023 3185 2003 2307 1012  102    0    0    0    0    0]

In [16]:
classifier_model.summary()

StatementMeta(PerforTestSpark, 0, 16, Finished, Available)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [17]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_data).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 2e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

StatementMeta(PerforTestSpark, 0, 17, Finished, Available)

In [18]:
classifier_model.compile(optimizer=optimizer,
                        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                        metrics = tf.metrics.BinaryAccuracy())

StatementMeta(PerforTestSpark, 0, 18, Finished, Available)

In [19]:
#tf.config.run_functions_eagerly(True)
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_data,
                               validation_data=valid_data,
                               epochs=epochs)

StatementMeta(PerforTestSpark, 0, 19, Submitted, Running)

Training model with https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/2
Epoch 1/5
1400/1400 [==============================] - 5253s 4s/step - loss: 0.5034 - binary_accuracy: 0.7863 - val_loss: 0.4421 - val_binary_accuracy: 0.86
Epoch 2/5
1400/1400 [==============================] - 5334s 4s/step - loss: 0.4223 - binary_accuracy: 0.8884 - val_loss: 0.4399 - val_binary_accuracy: 0.87
Epoch 3/5
1400/1400 [==============================] - 5438s 4s/step - loss: 0.4013 - binary_accuracy: 0.9107 - val_loss: 0.4374 - val_binary_accuracy: 0.87
Epoch 4/5
1400/1400 [==============================] - 5519s 4s/step - loss: 0.3851 - binary_accuracy: 0.9268 - val_loss: 0.4338 - val_binary_accuracy: 0.87
Epoch 5/5
1400/1400 [==============================] - 5562s 4s/step - loss: 0.3723 - binary_accuracy: 0.9405 - val_loss: 0.4304 - val_binary_accuracy: 0.8816

In [20]:
loss, accuracy = classifier_model.evaluate(test_data)
Y_pred = classifier_model.predict(test_data)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

StatementMeta(, , , Waiting, )

500/500 [==============================] - 563s 1s/step - loss: 0.4229 - binary_accuracy: 0.88
Loss: 0.4229238033294678
Accuracy: 0.8891249895095825

In [21]:
def step_fn(x) :
    if x <= 0:
        return 0
    if x > 0:
        return 1 

StatementMeta(, , , Waiting, )

In [22]:
y_pred = [step_fn(y) for y in Y_pred] 
print('Confusion Matrix')
print(confusion_matrix(np.concatenate([y for x, y in test_data], axis=0), y_pred))

StatementMeta(, , , Waiting, )

Confusion Matrix
[[3545  510]
 [ 385 3560]]